## Part 1: Obtain Entries from SQL Database

In [11]:
import pymysql
import pandas as pd 

In [12]:
conn = pymysql.connect(host='50.116.46.175',
                      user = 'diig_user',
                      password = 'vL6z3dVRxJbXshBj54og',
                      db = 'theproducebox',
                      cursorclass = pymysql.cursors.DictCursor) 

In [13]:
# get all users from the menu list
menulist_users = pd.read_sql_query("SELECT * from user where user_status_id='5' or user_status_id='9' or user_status_id='11'", conn)

In [14]:
# retain variables of interest
interest = ['user_id', 'fullname', 'user_email', 'signup_date', 'menu_box_type_id', 'ltv_total_order_count', 'ltv_total_sales', 'is_customer']
menulist_users = menulist_users[interest]

In [15]:
# obtain last date of purchase
last_order = []
for user in menulist_users['user_id']:
    sql_call = f"SELECT max(date) FROM `order` where user_id={user}"
    df = pd.read_sql_query(sql_call, conn)
    if(len(df)==0):
        last_order.append(None)
    else:
        last_order.append(df['max(date)'][0]);

In [16]:
# add column for last order date
menulist_users["last_order"] = last_order

## Part 2: RFM Analysis

In [29]:
import numpy as np
from datetime import datetime, timedelta, date

In [30]:
menulist_users['signup_date'] = pd.to_datetime(menulist_users['signup_date'])
menulist_users['last_order'] = pd.to_datetime(menulist_users['last_order'])

In [31]:
# separate new users 
new_user_indexes = []
index = 0
for date in menulist_users['signup_date']:
    if (datetime.now() - date).days <= 31:
        new_user_indexes.append(index)
    index+=1

In [32]:
df = menulist_users
df_new_users = df.iloc[new_user_indexes,:]
df = df.drop(new_user_indexes)

# df_new_users contains all the people that have signed up in the past 30 days
# df contains everyone else

In [33]:
#create a generic user dataframe to keep CustomerID and new segmentation scores
seg_user = pd.DataFrame(df['user_id'].unique())
seg_user.columns = ['user_id']

In [34]:
# recency
user_last_order = df[["user_id", "last_order"]]
user_last_order.columns = ["user_id", "last_order"]

recency = []
for row in user_last_order['last_order']:
    if(pd.isnull(row)):
        recency.append(5000) # arbitrarily assign a high number of days to those that have never purchased
    else:
        recency.append((datetime.now()- row).days) 

user_last_order["recency"] = recency

#merge this dataframe to our new user dataframe
seg_user = pd.merge(seg_user, user_last_order[['user_id','recency']], on='user_id')

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [35]:
#build 5 clusters for recency and add it to dataframe

from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=5)
kmeans.fit(seg_user[['recency']])
seg_user['recency_cluster'] = kmeans.predict(seg_user[['recency']])

#function for ordering cluster numbers
def order_cluster(cluster_field_name, target_field_name,df,ascending):
    new_cluster_field_name = 'new_' + cluster_field_name
    df_new = df.groupby(cluster_field_name)[target_field_name].mean().reset_index()
    df_new = df_new.sort_values(by=target_field_name,ascending=ascending).reset_index(drop=True)
    df_new['index'] = df_new.index
    df_final = pd.merge(df,df_new[[cluster_field_name,'index']], on=cluster_field_name)
    df_final = df_final.drop([cluster_field_name],axis=1)
    df_final = df_final.rename(columns={"index":cluster_field_name})
    return df_final

seg_user = order_cluster('recency_cluster', 'recency',seg_user,False)

In [36]:
# frequency
#get order counts for each user and create a dataframe with it
user_frequency = df[["user_id","ltv_total_order_count"]]
user_frequency.columns = ['user_id','frequency']

seg_user = pd.merge(seg_user, user_frequency, on='user_id')

In [37]:
#k-means
kmeans = KMeans(n_clusters=5)
kmeans.fit(seg_user[['frequency']])
seg_user['frequency_cluster'] = kmeans.predict(seg_user[['frequency']])

#order the frequency cluster
seg_user = order_cluster('frequency_cluster', 'frequency',seg_user,True)

#see details of each cluster
seg_user.groupby('frequency_cluster')['frequency'].describe()

,count,mean,std,min,25%,50%,75%,max
frequency_cluster,,,,,,,,
0,5318.0,10.105303,8.708031,0.0,4.0,7.0,16.0,31.0
1,2341.0,52.056386,13.559028,32.0,40.0,50.0,64.0,78.0
2,1664.0,105.382212,16.402904,79.0,91.0,105.0,119.0,135.0
3,1179.0,166.123834,19.693964,136.0,148.0,165.0,183.0,203.0
4,956.0,241.594142,24.427487,204.0,220.0,239.0,262.0,298.0


In [38]:
#calculate revenue for each customer
user_revenue = df[["user_id", "ltv_total_sales"]]
user_revenue.columns = ['user_id','revenue']

#merge it with our main dataframe
seg_user = pd.merge(seg_user, user_revenue, on='user_id')

In [39]:
#apply clustering
kmeans = KMeans(n_clusters=5)
kmeans.fit(seg_user[['revenue']])
seg_user['revenue_cluster'] = kmeans.predict(seg_user[['revenue']])


#order the cluster numbers
seg_user = order_cluster('revenue_cluster', 'revenue',seg_user,True)

#show details of the dataframe
seg_user.groupby('revenue_cluster')['revenue'].describe()

,count,mean,std,min,25%,50%,75%,max
revenue_cluster,,,,,,,,
0,6573.0,538.410338,465.558400,0.00,157.8200,371.600,858.3000,1695.84
1,2650.0,2850.549823,754.435353,1697.73,2171.0100,2770.610,3476.4100,4322.41
2,1593.0,5784.422856,969.070479,4326.05,4932.9900,5699.530,6590.3600,7707.38
3,570.0,9637.435614,1709.413708,7721.93,8279.4000,9089.740,10530.4175,14625.56
4,72.0,19656.962778,4647.158876,14756.72,16359.2325,18161.475,22586.2325,37559.03


In [40]:
seg_user['average_fm'] = (seg_user['frequency_cluster'] + seg_user['revenue_cluster']) / 2.0

In [41]:
# segments stored in list seg
new_user_list = df_new_users['user_id'].tolist()
seg = []
for index, row in menulist_users.iterrows():
    if row['user_id'] in new_user_list:
        seg.append("New Customer")
    else:
        seg_row = seg_user.loc[seg_user['user_id'] == row['user_id']]
        if ((seg_row['recency_cluster'] == 4) & (seg_row['average_fm'] >= 3.5)).bool():
            seg.append('Champion')
        elif ((seg_row['recency_cluster'] == 4) & (seg_row['average_fm'] >= 3)).bool():
            seg.append('Loyal Customer')
        elif ((seg_row['recency_cluster'] == 4) & (seg_row['average_fm'] >= 2)).bool():
            seg.append('Potential Loyalist')
        elif ((seg_row['recency_cluster'] == 4) & (seg_row['average_fm'] < 2)).bool():
            seg.append('Promising')
        elif ((seg_row['recency_cluster'] == 3) & (seg_row['average_fm'] >= 2)).bool():
            seg.append('Customers Needing Attention')
        elif (((seg_row['recency_cluster'] >= 0) & (seg_row['recency_cluster'] <= 3)) & (seg_row['average_fm'] >= 3)).bool():
            seg.append('Once High Value')     
        else:
            seg.append('Lost')

### Final DataFrame
After running the next code chunk, the dataframe user_segments will contain one column with user ID and one column with the segment the user is in.

In [43]:
# join user id and segments into new dataframe
user_segments = pd.DataFrame(list(zip(menulist_users['user_id'].tolist(), seg)),
              columns=['user_id','segment'])
user_segments

,user_id,segment
0,6,Potential Loyalist
1,10,Promising
2,12,Customers Needing Attention
3,13,Champion
4,17,Loyal Customer
...,...,...
13626,61337,New Customer
13627,61384,New Customer
13628,61440,New Customer
13629,61473,New Customer
